# This Script is Scraping All Data Analyst jobs from Wuzzuf Website

### Importing Libraries

In [32]:
from bs4 import BeautifulSoup as bs
import requests
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import time
from urllib.request import urlopen
import csv



###  Creating Spreadsheet and Adding Header

In [33]:
file = open('D:\Wuzzuf_Data_analyst.csv', 'w')
header=["Job_Title", "Company_Name", "Company_Address", "Job_Time", "Job_Level", "Publish_Date","Job_Link"]
with open('D:\Wuzzuf_Data_analyst.csv', 'a',encoding="utf-8",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)


### Creating Function to Scrap The Passed URL

In [34]:
def UrlScrape(url):
    #Sending request to the URL top open it
    client=urlopen(url)
    #Redaing HTML Content
    htmlcontent=client.read()
    #Close Connection with URL
    client.close
    #Parsing HTML Content to Organize it and be Look Understandable
    htmlparsed = bs(htmlcontent,"html.parser")
    #Finding all page sections which contain job details
    jobscontainer=htmlparsed.find_all("div",{"class":"css-pkv5jc"})
    #loop on all page jobs data and extracting the job title, company name, company address, job time, job level and the period of time which the job posted then converting it to post date
    for container in jobscontainer:
        job_title=container.find("h2",{"class":"css-m604qf"})
        job_title=job_title.text.strip()
        company_name=container.find("a",{"class","css-17s97q8"})
        company_name=company_name.text.replace('-','').strip()
        company_address=container.find("span",{"class","css-5wys0k"})
        company_address=company_address.text.replace(',','\\').strip()
        job_time=container.find("a",{"class","css-n2jc4m"})
        job_time=job_time.text.strip()
        job_level=container.find_all("a",{"class","css-o171kl"})
        job_level=job_level[1].text.strip()
        job_link=container.find("a",{"class","css-o171kl"}).get('href')
        job_link="https://wuzzuf.net"+job_link
        #Converting the period of time which the job posted to it's date by subtracting this period from today date
        publich_date=container.find("div",{"class","css-4c4ojb"})
        if publich_date==None:
            publich_date=container.find("div",{"class","css-do6t5g"})
        publich_date=publich_date.text.strip()
        if ("days" in publich_date) or ("day" in publich_date):
            day=publich_date[0:2]
            datediff = date.today() + datetime.timedelta(days = -int(day))
            pub_date=str(datediff.day)+'/'+str(datediff.month)+'/'+str(datediff.year)
        elif ("month" in publich_date) or ("months" in publich_date):
            month=publich_date[0:2]
            datediff = date.today() + relativedelta(months = -int(month))
            pub_date=str(datediff.day)+'/'+str(datediff.month)+'/'+str(datediff.year)
        elif ("year" in publich_date) or ("years" in publich_date):
            year=publich_date[0:2]
            datediff = date.today() + relativedelta(years = -int(year))
            pub_date=str(datediff.day)+'/'+str(datediff.month)+'/'+str(datediff.year)
        else:
            pub_date=str(date.today().day)+'/'+str(date.today().month)+'/'+str(date.today().year)
        row=[job_title,company_name,company_address,job_time,job_level,pub_date,job_link]
        #Appending Job Data to Spreadsheet
        with open('D:\Wuzzuf_Data_analyst.csv', 'a',encoding="utf-8",newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row)
    
    

In [35]:
#Looping on All Wuzzuf Search Result Pages of Data Analyst by incrementing the page index in the URL
#11 is the number of pages that contain the search results on wuzzuf website
for i in range (0,11):
    UrlScrape('https://wuzzuf.net/search/jobs/?a=navbl%7Cspbl&q=data%20analyst&start='+str(i))
    
    

In [36]:
file.close()
